In [75]:
from datasets import load_dataset

dataset = load_dataset("esnli")

Reusing dataset esnli (C:\Users\lenna\.cache\huggingface\datasets\esnli\plain_text\0.0.2\a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc)


  0%|          | 0/3 [00:00<?, ?it/s]

In [76]:
from transformers import AutoTokenizer, BertTokenizer

max_len = len(max(dataset['train']['explanation_1']))
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", model_max_length=max_len)


def tokenize_function(examples):
    return tokenizer(examples["explanation_1"], padding='max_length', truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at C:\Users\lenna/.cache\huggingface\transformers\23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.16.2",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at C:\Users\lenna/.cache\huggingface\transformers\0e1bbfda7f6

  0%|          | 0/10 [00:00<?, ?ba/s]

Loading cached processed dataset at C:\Users\lenna\.cache\huggingface\datasets\esnli\plain_text\0.0.2\a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc\cache-7009e3b628105c5a.arrow


In [77]:
# Selected smaller datasets to test out if it works
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Loading cached shuffled indices for dataset at C:\Users\lenna\.cache\huggingface\datasets\esnli\plain_text\0.0.2\a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc\cache-c9bf9bb97a374f0e.arrow
Loading cached shuffled indices for dataset at C:\Users\lenna\.cache\huggingface\datasets\esnli\plain_text\0.0.2\a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc\cache-3fe858b6695348a5.arrow


In [90]:
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at C:\Users\lenna/.cache\huggingface\transformers\23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Model config BertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dim": 768,
  "dropout": 0.1,
  "hidden_act": "gelu",
  "hidden_dim": 3072,
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_nor

In [79]:
from transformers import TrainingArguments

# I set the batch sizes a little higher so that training wouldn't take decades
training_args = TrainingArguments(output_dir="test_trainer", per_device_train_batch_size=100, per_device_eval_batch_size=100)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [80]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [81]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [82]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [83]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: explanation_3, premise, hypothesis, explanation_1, explanation_2.
c:\users\lenna\appdata\local\programs\python\python38\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 100
  Total train batch size (w. parallel, distributed & accumulation) = 100
  Gradient Accumulation steps = 1
  Total optimization steps = 30


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=30, training_loss=1.1604153951009115, metrics={'train_runtime': 5059.361, 'train_samples_per_second': 0.593, 'train_steps_per_second': 0.006, 'total_flos': 192710022750000.0, 'train_loss': 1.1604153951009115, 'epoch': 3.0})

In [86]:
small_validation_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(1000))

In [87]:
trainer.predict(small_validation_dataset)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: explanation_3, premise, hypothesis, explanation_1, explanation_2.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 100


PredictionOutput(predictions=array([[-0.40270138, -0.6136879 , -0.5145344 ],
       [-0.50355315, -0.4896132 , -0.5198801 ],
       [-0.36529583, -0.64300907, -0.5176122 ],
       ...,
       [-0.4442964 , -0.55423146, -0.5170429 ],
       [-0.4353521 , -0.5774934 , -0.5218556 ],
       [-0.41022757, -0.6020552 , -0.5180179 ]], dtype=float32), label_ids=array([2, 1, 0, 0, 1, 2, 2, 1, 1, 1, 1, 0, 1, 0, 2, 2, 2, 0, 2, 2, 0, 2,
       0, 2, 1, 0, 2, 0, 0, 1, 1, 2, 0, 1, 0, 1, 0, 2, 1, 1, 2, 0, 1, 1,
       1, 2, 2, 1, 0, 1, 0, 0, 2, 0, 1, 0, 2, 2, 2, 0, 1, 0, 0, 0, 2, 0,
       2, 2, 0, 0, 2, 0, 0, 0, 2, 1, 0, 1, 2, 1, 1, 0, 2, 2, 1, 0, 2, 1,
       2, 0, 2, 0, 1, 2, 0, 1, 2, 0, 2, 1, 1, 2, 2, 2, 2, 2, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 2, 1, 0, 1, 0, 1, 1, 2, 0, 1, 1, 2, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 2, 0, 0, 2, 0, 2, 0, 1, 1, 0, 0, 1, 1, 0, 2, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 2, 2, 2, 1, 0, 2, 2, 1, 0, 1, 0, 2, 1, 0, 0, 2,
       2, 0, 1, 0, 0, 1, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 1, 1,